In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm.notebook import tqdm
from pathlib import Path
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.utils import shuffle

import tensorflow as tf

from importlib import reload
import sentinel_utils
import keras_model_creator
import plot_utils

from data_generator import DataGenerator

pd.options.mode.copy_on_write = True

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
# %load_ext tensorboard
# %tensorboard --logdir=$tensorboard_dir
# tensorboard_dir = str(model_dir.joinpath('tensorboard_logs'))

In [3]:
sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']

band_indices = list(range(len(sentinel_bands)))

In [4]:
loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

fixed_params = dict(
    shards_dir=shards_dir,
    band_indices=band_indices,
    loss=loss,
    batch_size=batch_size,
    base_filters=base_filters,
    dropout=0.2,
    epochs=11,
    overwrite=False,
    verbose=1,
    print_log=False
)

Select the season(s) to use and calculate the mean and standard deviation for each band if required. These are used to normalise the batches.

Select the classes to use based on minimum occorrences. This also removes labels that do not have any selected classes.

In [ ]:
reload(sentinel_utils)
reload(keras_model_creator)
utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

all_seasons_names = ['spring', 'summer', 'autumn', 'winter']
all_seasons = ['03', '06', '09', '12']
season_combinations = list(itertools.chain.from_iterable(
    itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1)
))

model_parent_dir = Path('models', 'season_selection')
for seasons in [all_seasons]:
# for seasons in (pbar := tqdm(season_combinations)):
    # pbar.set_description('-'.join(seasons))

    selected_classes = utils.get_processed_labels()
    data_summary = utils.get_data_summary(
        shards_dir, seasons, selected_classes)
                   
    model_dir = model_parent_dir.joinpath(
        f'{loss}-{len(selected_classes.index)}'
        f'-{selected_classes.shape[1]}-{len(band_indices)}'
        f'-{"_".join(seasons)}-{batch_size}-{base_filters}-lstm'
    )
    model_dir.mkdir(parents=True, exist_ok=True)

    changing_params = dict(
        selected_classes=selected_classes,
        model_dir=model_dir,
        seasons=seasons,
        data_summary=data_summary
    )
    params = fixed_params | changing_params
    keras_model_creator.KerasModelCreator(**params).run()

Epoch 1/11


I0000 00:00:1722156123.287430   21880 service.cc:145] XLA service 0x7fe800001bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722156123.287473   21880 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1722156142.341086   21880 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 28 bytes spill stores, 4 bytes spill loads

I0000 00:00:1722156142.364693   21880 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.4684 - auc: 0.8861 - loss: 0.3481 - macrof1score: 0.4325 - microf1score: 0.6284 - prc: 0.7308 - precision: 0.7381 - recall: 0.5496 - weightedf1score: 0.5903

I0000 00:00:1722157879.738453   24106 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1287', 308 bytes spill stores, 308 bytes spill loads

I0000 00:00:1722157879.773642   24108 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1287', 192 bytes spill stores, 192 bytes spill loads




Epoch 1: val_recall improved from 0.50000 to 0.63527, saving model to models/season_selection/binary_crossentropy-237212-7-10-03_06_09_12-64-32-lstm/model.keras
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 1841s 511ms/step - accuracy: 0.4684 - auc: 0.8861 - loss: 0.3481 - macrof1score: 0.4325 - microf1score: 0.6285 - prc: 0.7308 - precision: 0.7381 - recall: 0.5496 - weightedf1score: 0.5903 - val_accuracy: 0.5209 - val_auc: 0.9257 - val_loss: 0.2908 - val_macrof1score: 0.5361 - val_microf1score: 0.7099 - val_prc: 0.8173 - val_precision: 0.8044 - val_recall: 0.6353 - val_weightedf1score: 0.6815 - learning_rate: 1.0000e-04
Epoch 2/11
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.5281 - auc: 0.9225 - loss: 0.2946 - macrof1score: 0.5412 - microf1score: 0.7103 - prc: 0.8050 - precision: 0.7799 - recall: 0.6522 - weightedf1score: 0.6821
Epoch 2: val_recall improved from 0.63527 to 0.65054, saving model to models/season_selection/binary_crossentropy-237212-7-10-03_06_09_12-64-32-lstm/model.ke

Evaluate the model for given years and save the results in the model's directory.

In [ ]:
reload(plot_utils)

best_metric = 'val_weightedf1score'
best_rows = []

for seasons in list(season_combinations):
    model_dirs = list(model_parent_dir.glob(f'*-{"_".join(seasons)}-*'))
    
    for model_dir in reversed(model_dirs):
        df = pd.read_csv(model_dir.joinpath('model.log'))
        best = df[df[best_metric] == df[best_metric].max()]
        

        if 'lstm' in str(model_dir):
            best[all_seasons_names] = [int(s in seasons)*2 for s in all_seasons]
        else:
            best[all_seasons_names] = [int(s in seasons) for s in all_seasons]
            
        best_rows.append(best)
best_df = pd.concat(best_rows).round(2).reset_index(drop=True)
plot_utils.PlotUtils().line_heatmap(
    best_df, all_seasons_names, [0.2, 0.8]
)

In [ ]:
# for eval_year in [2017, 2020, 2021, 2022, 2023]:
    # eval_generator = DataGenerator(
        # utils.selected_classes.index, shuffle=False, year=eval_year, **params)
#     preds_path = model_dir.joinpath(f"preds_{params['year']}.npy")
#     if preds_path.is_file():
#         y_pred = np.load(preds_path)
#     else:
#         y_pred = model.predict(x=eval_generator, verbose=1)
#         np.save(preds_path, y_pred)

#     y_true = utils.selected_classes.iloc[:y_pred.shape[0]].to_numpy()
#     cm = confusion_matrix(y_true.flatten(), (y_pred > 0.5).flatten().astype(int))
#     plot = ConfusionMatrixDisplay(confusion_matrix=cm).plot()
#     display(plot)

Visualise the confusion matrix for each class.

In [ ]:
# import matplotlib.pyplot as plt

# class_names = utils.selected_classes.columns

# f, axes = plt.subplots(4, 2, figsize=(25, 30))
# axes = axes.ravel()
# for label in range(y_true.shape[1]):
#     cm = confusion_matrix(y_true[..., label].astype(int), (y_pred[..., label] > 0.5).astype(int))
#     disp = ConfusionMatrixDisplay(cm)
#     disp.plot(ax=axes[label], values_format='.4g')
#     disp.ax_.set_title(f'{class_names[label]}')
#     if label < 25:
#         disp.ax_.set_xlabel('')
#     if label % 5 != 0:
#         disp.ax_.set_ylabel('')
#     disp.im_.colorbar.remove()

# plt.subplots_adjust(wspace=0.2, hspace=0.001)
# f.colorbar(disp.im_, ax=axes)
# plt.show()


In [ ]:
# import tensorflow
# tensorflow.keras.utils.plot_model(model, show_shapes=True)
# model.summary()

Visualise the model.

In [ ]:
# import tensorflow
# tensorflow.keras.utils.plot_model(
#     model, to_file=model_dir.joinpath('model.png'),
#     show_shapes=True, show_layer_activations=True,
# )

Correlate the model with climate variables (to move?)

In [ ]:
# import plotly.express as px

# class_names = utils.selected_classes.columns
# indices = utils.selected_classes.index

# preds_path = model_dir.joinpath(f'preds_2020.npy')
# y_prev = np.load(preds_path)

# weather_prev = pd.read_csv(Path('weather_data', 'era5_2020.csv'))
# eval_years = [2021, 2022, 2023]
# corrs = []

# for eval_year in eval_years:
#     preds_path = model_dir.joinpath(f'preds_{eval_year}.npy')
#     y_pred = np.load(preds_path)
#     y_diff = pd.DataFrame(y_pred - y_prev, columns=class_names)
    
#     weather = pd.read_csv(Path('weather_data', f'era5_{eval_year}.csv'))
#     weather_diff = ((weather - weather_prev)
#                     .loc[indices]
#                     .iloc[:y_pred.shape[0]]
#                     .iloc[y_diff.index])
    
#     corr = y_diff.join(weather_diff).corr(method='pearson').round(2)
#     corrs.append(corr)

#     y_prev = y_pred
#     weather_prev = weather

# fig = px.imshow(
#     np.array(corrs),
#     animation_frame=0,
#     labels=dict(color="Corr coef"),
#     x=corrs[0].index,
#     y=corrs[0].columns,
#     title='Annual correlation heatmap',
#     text_auto=True, aspect='auto', zmin=0, height=500
# )
# fig.layout.sliders[0]['currentvalue']['prefix'] = ''
# for year, step in zip(eval_years, fig.layout.sliders[0].steps):
#     step.label = str(year)

# fig